https://www.kaggle.com/datasets/dimka11/cl-cup-it-2023-embs

KNN, Linear Regression, SVC классификация на основе эмбедингов из Sentence Transformers 
Модель - 

In [1]:
# !pip install scikit-learn-intelex

In [1]:
from sklearnex import patch_sklearn 

patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f752f955d8b180d6db00ba68da5adc60025dad11ab2ba7bd2738b5ae65047404
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers


In [36]:
import numpy as np
import pandas as pd
import os

from sklearn.metrics import ndcg_score, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import RidgeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer

import umap
from tqdm import tqdm

from scipy.spatial import distance

# from catboost import CatBoostRanker # https://colab.research.google.com/github/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb#scrollTo=4pr7PqFuNdYo
# https://www.kaggle.com/code/kooaslansefat/proof-of-concept-catboost-ranker
# https://www.kaggle.com/code/cafelatte1/catboost-ranker-training/notebook

In [4]:
train = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_train.jsonl', lines=True)
test = pd.read_json('/kaggle/input/cl-cup-it-2023-ds/CL_Cup IT_Data_Scince___VK_/ranking_test.jsonl', lines=True)

Look at the examples:

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88107 entries, 0 to 88106
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      88107 non-null  object
 1   comments  88107 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [ ]:
test.info()

In [ ]:
train.head(5)

In [ ]:
train.iloc[0, 1][3]['text']

In [ ]:
train.iloc[0, 1][3]['score']

In [ ]:
test.head(5)

In [ ]:
test.loc[0,'comments']

In [ ]:
train.explode('comments').head(20)

In [ ]:
train.explode('comments')['comments'].str['score']

Metric Example

In [10]:
true_relevance = [1, 0, 2]
scores = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]

# Convert to one-hot
values = [1, 0, 2]
n_values = np.max(values) + 1
np.eye(n_values)[values]

y_true = [[0, 1, 0], [1, 0, 0], [0, 0, 1]]
y_score = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
ndcg_score(y_true, y_score, k=2)

## Prepare train & test

In [5]:
def prepare_df(train, test=False):
    train = train.explode('comments')
    if not test:
        train['score'] = train['comments'].str['score']
    train['comment'] = train['comments'].str['text']
    train = train.drop(columns=['comments'])
    train['text_id'] = train.index
    return train
train = prepare_df(train)
scores = train['score'].values
test = prepare_df(test, True)
train.head(20)

,text,score,comment,text_id
0,How many summer Y Combinator fundees decided n...,0,Going back to school is not identical with giv...,0
0,How many summer Y Combinator fundees decided n...,1,There will invariably be those who don't see t...,0
0,How many summer Y Combinator fundees decided n...,2,For me school is a way to be connected to what...,0
0,How many summer Y Combinator fundees decided n...,3,I guess it really depends on how hungry you ar...,0
0,How many summer Y Combinator fundees decided n...,4,I know pollground decided to go back to school...,0
1,CBS acquires last.fm for $280m,0,It will be curious to see where this heads in ...,1
1,CBS acquires last.fm for $280m,1,Does this mean that there's now a big-name com...,1
1,CBS acquires last.fm for $280m,2,Also on BBC News: http://news.bbc.co.uk/1/low...,1
1,CBS acquires last.fm for $280m,3,I don't understand what they do that is worth ...,1
1,CBS acquires last.fm for $280m,4,sold out too cheaply. given their leadership p...,1


## Get Embs from ST

In [15]:
model_name = 'paraphrase-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

In [6]:
get_embs = False

if get_embs:
    batch_size = 256
    train_text_embs = model.encode(train.groupby('text_id').first()['text'].values, batch_size=batch_size)
    test_text_embs = model.encode(test.groupby('text_id').first()['text'].values, batch_size=batch_size)

    train_comment_embs = model.encode(train['comment'].values, batch_size=batch_size)
    test_comment_embs = model.encode(test['comment'].values, batch_size=batch_size)

In [7]:
if get_embs:
    np.save("train_text_embs.npy", train_text_embs)
    np.save("test_text_embs.npy", test_text_embs)
    np.save("train_comment_embs.npy", train_comment_embs)
    np.save("test_comment_embs.npy", test_comment_embs)
else:
    train_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_text_embs.npy' ,allow_pickle=True)
    test_text_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_text_embs.npy' ,allow_pickle=True)
    train_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/train_comment_embs.npy' ,allow_pickle=True)
    test_comment_embs = np.load('/kaggle/input/cl-cup-it-2023-embs/paraphrase-MiniLM-L6-v2/paraphrase-MiniLM-L6-v2/test_comment_embs.npy' ,allow_pickle=True)

Следующую ячейку запускать ровно один раз, 

или переименовывать перемнные

In [78]:
concat_text_comments = True

if concat_text_comments:
    train_comment_embs = np.concatenate([np.repeat(train_text_embs, 5, 0), train_comment_embs], axis=1)
    test_comment_embs = np.concatenate([np.repeat(test_text_embs, 5, 0), test_comment_embs], axis=1)

## Clusters

Данные не кластеризуются ни фига

https://umap-learn.readthedocs.io/en/latest/basic_usage.html

In [79]:
get_clusters = False

if get_clusters:
    number_of_examples = 1000

    reducer = umap.UMAP(n_neighbors=30,
            min_dist=0.1,
            n_components=2,
            metric='euclidean')

    X_embs = StandardScaler().fit_transform(train_comment_embs[0:number_of_examples])
    embedding = reducer.fit_transform(X_embs)

    plt.scatter(embedding[:, 0], embedding[:, 1], c=scores[0:number_of_examples], cmap='Spectral', s=5)
    plt.gca().set_aspect('equal', 'datalim')
    plt.colorbar(boundaries=np.arange(6)-0.5).set_ticks(np.arange(5))
    plt.title('', fontsize=24);

## Split

In [84]:
splitter = GroupShuffleSplit(test_size=.20, n_splits=1, random_state = 11)
split = splitter.split(train_comment_embs, groups=train['text_id'])
train_inds, test_inds = next(split)

X_train = train_comment_embs[train_inds]
X_test = train_comment_embs[test_inds]
y_train = scores[train_inds]
y_test = scores[test_inds]

## KNN

Берем часть данные для ускорения работы

In [88]:
# X_train_part = X_train[0:352425//3]
# y_train_part = y_train[0:352425//3]
X_train_part = X_train[0:352425]
y_train_part = y_train[0:352425]

In [117]:
# neigh = KNeighborsClassifier(n_neighbors=18, metric='cosine', n_jobs=4)
neigh = KNeighborsClassifier(n_neighbors=1, metric='cosine', n_jobs=4)
neigh.fit(X_train_part, y_train_part)

KNeighborsClassifier(metric='cosine', n_jobs=4, n_neighbors=1)

In [ ]:
y_pred = neigh.predict(X_test)
y_pred_proba = neigh.predict_proba(X_test)
accuracy_score(y_test, y_pred) # 0.22470775  knn 18

Нулевой класс предсказывается относительно неплохо.

0 - класс предсказыватеся относительно хорошо, надеюсь нигде не было лика

In [69]:
accuracy_score(y_test[np.where(y_test == 0)],  y_pred[np.where(y_test == 0)]) # 0.4752014527

0.4752014527295426

ndcg - не уверен, что она правильно тут считается

In [21]:
y_test_ = np.eye(5)[y_test]
ndcg_score(y_test_ ,y_pred_proba)

0.6062911695239217

In [18]:
# ndcg = 0
# shift = 5

# for i in tqdm(range(y_test.shape[0] //5)):
#     n_values = 5
#     y_test_ = np.eye(n_values)[y_test[i*5:shift]]
#     ndcg += ndcg_score(y_test_, y_pred_proba[i*5:shift])
#     shift += 5

# ndcg = ndcg / (y_test.shape[0] // 5)
# print(ndcg)

100%|██████████| 17622/17622 [00:11<00:00, 1494.36it/s]

0.6052703379693739


## Определяем близость коментария к тексту и ранжируем коменты относительно расстояния

In [81]:
scores_all = np.zeros(train_comment_embs.shape[0])
shift = 5
for i, item in tqdm(enumerate(train_text_embs), total=train_text_embs.shape[0]):
    distances = np.zeros(5)
    
    distances[0] = distance.cosine(item, train_comment_embs[i*5])
    distances[1] = distance.cosine(item, train_comment_embs[(i*5)+1])
    distances[2] = distance.cosine(item, train_comment_embs[(i*5)+2])
    distances[3] = distance.cosine(item, train_comment_embs[(i*5)+3])
    distances[4] = distance.cosine(item, train_comment_embs[(i*5)+4])
    
    scores_all[i*5:i*5+5] = np.argsort(distances)

  0%|          | 0/88107 [00:00<?, ?it/s]


ValueError: operands could not be broadcast together with shapes (384,) (768,) 

In [54]:
accuracy_score(train['score'], scores_all)

0.22290169907044843

In [67]:
train_scores = train['score'].values
train_scores[np.where(train_scores == 0)]

array([0, 0, 0, ..., 0, 0, 0])

Предсказания 1-го класса

In [105]:
accuracy_score(train['score'].values[np.where(train['score'].values == 0)],  scores_all[np.where(train['score'].values == 0)])

1.0

ndcg

In [93]:
y_test_ = np.eye(5)[train['score'].values]
scores_all_ = np.eye(5)[scores_all.astype(int)]
ndcg_score(y_test_, scores_all_)

0.601437766748415

## Scaler

In [101]:
sc = StandardScaler()
X_train_part = sc.fit_transform(X_train_part)
X_test = sc.transform(X_test)

## Linear regression

Регрессия и окруление значений

In [106]:
reg = LinearRegression().fit(X_train_part, y_train_part)

In [107]:
y_pred = reg.predict(X_test)

accuracy_score(y_test, np.round(y_pred))

0.2177618885484054

## Ridge

In [102]:
ridge_clf = RidgeClassifier(alpha=0.9)
ridge_clf.fit(X_train_part, y_train_part)
y_pred = ridge_clf.predict(X_test)

d = ridge_clf.decision_function(X_test)
y_pred_proba = np.exp(d) / np.sum(np.exp(d))

accuracy_score(y_test, y_pred)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.61978e-08): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


0.2845760980592441

In [111]:
accuracy_score(y_test[np.where(y_test == 0)],  y_pred[np.where(y_test == 0)])

0.6546362501418681

In [112]:
y_test_ = np.eye(5)[y_test]
ndcg_score(y_test_ ,y_pred_proba)

0.6474772950887161

In [93]:
# lr = LogisticRegression(penalty = 'elasticnet', solver = 'saga', l1_ratio = 0.5) # The Elastic-Net regularization is only supported by the ‘saga’ solver.
lr = LogisticRegression(l1_ratio = 0.5)
lr.fit(X_train_part, y_train_part)
y_pred = lr.predict(X_test)
accuracy_score(y_test, y_pred)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1479: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  "(penalty={})".format(self.penalty)
/opt/conda/lib/python3.7/site-packages/daal4py/sklearn/linear_model/logistic_path.py:552: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.28374758824197027

In [103]:
# regr = ElasticNet(random_state=0, alpha=0.5, l1_ratio=0.5)
# regr.fit(X_train_part, y_train_part)
# y_pred = reg.predict(X_test)
# accuracy_score(y_test, np.round(y_pred))

0.217739189649302

## LDA

In [113]:
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X_train_part, y_train_part)
y_pred = lda_clf.predict(X_test)
y_pred_proba = lda_clf.predict_proba(X_test)

accuracy_score(y_test, y_pred), accuracy_score(y_test[np.where(y_test == 0)],  y_pred[np.where(y_test == 0)])

(0.284916581545795, 0.6128702757916241)

In [114]:
y_test_ = np.eye(5)[y_test]
ndcg_score(y_test_ ,y_pred_proba)

0.6482390003133851

## MLP

In [116]:
clf_mlp = MLPClassifier(random_state=1, max_iter=50, verbose=True) # hidden_layer_sizes
clf_mlp.fit(X_train_part, y_train_part)
y_pred = clf_mlp.predict(X_test)
y_pred_proba = clf_mlp.predict_proba(X_test)

accuracy_score(y_test, y_pred), accuracy_score(y_test[np.where(y_test == 0)],  y_pred[np.where(y_test == 0)])

Iteration 1, loss = 1.59951292
Iteration 2, loss = 1.55587718
Iteration 3, loss = 1.55247946
Iteration 4, loss = 1.54983255
Iteration 5, loss = 1.54684025
Iteration 6, loss = 1.54377250
Iteration 7, loss = 1.54072054
Iteration 8, loss = 1.53690082
Iteration 9, loss = 1.53326066
Iteration 10, loss = 1.52944546
Iteration 11, loss = 1.52576680
Iteration 12, loss = 1.52173529
Iteration 13, loss = 1.51830824
Iteration 14, loss = 1.51460483
Iteration 15, loss = 1.51135521
Iteration 16, loss = 1.50758273
Iteration 17, loss = 1.50468242
Iteration 18, loss = 1.50191349
Iteration 19, loss = 1.49921001
Iteration 20, loss = 1.49660072
Iteration 21, loss = 1.49437965
Iteration 22, loss = 1.49217159
Iteration 23, loss = 1.48981270
Iteration 24, loss = 1.48779756
Iteration 25, loss = 1.48581619
Iteration 26, loss = 1.48393934
Iteration 27, loss = 1.48202840
Iteration 28, loss = 1.48057412
Iteration 29, loss = 1.47871104
Iteration 30, loss = 1.47719289
Iteration 31, loss = 1.47573318
Iteration 32, los

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


(0.2583588695948246, 0.41277948019521055)

In [ ]:
y_test_ = np.eye(5)[y_test]
ndcg_score(y_test_ ,y_pred_proba)

## SVC

Классификация

O(n_samples^2 * n_features)

Очень медленно даже с линейными кернелом

In [34]:
svc = SVC(kernel='linear', gamma='auto', C=1.0) # kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} or callable, default=’rbf’
svc.fit(X_train_part[0:10000], y_train_part[0:10000])

SVC(gamma='auto', kernel='linear')

In [38]:
y_pred = svc.predict(X_test)
accuracy_score(y_test, y_pred)

0.2362501418681194